In [34]:
import pandas as pd

with open("data.txt", "r") as f:
    lines = f.readlines()
data = [line.strip().split(",") for line in lines][:-1] 
df = pd.DataFrame(data, columns=["s_len", "s_wid", "p_len", "p_wid", "class"])

label_mapping = {
    "Iris-setosa": 0,
    "Iris-versicolor": 1, # colour hahah
    "Iris-virginica": 2
}

df['class'] = df['class'].map(label_mapping)
df = df.astype(float)
df

,s_len,s_wid,p_len,p_wid,class
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [35]:
import numpy as np

In [73]:
def relu(data):
    return np.clip(data, 0, np.abs(np.max(data)))

In [74]:
relu(np.random.normal(size=(3,3)))

array([[0.98157915, 0.        , 0.        ],
       [0.0490892 , 0.32503843, 0.55209061],
       [0.        , 0.80597677, 0.        ]])

In [80]:
class Neuron:
    def __init__(self, weight, bias, activation_func):
        self.weight = weight
        self.bias = bias
        self.activation_func = activation_func
    
    def __call__(self, data):
        h = np.sum(data) * self.weight + self.bias 
        if self.activation_func:
            h = self.activation_func(h)
        return h

In [85]:
n = Neuron(-.25, 4, relu)
n(df.iloc[0, :-1])

np.float64(1.4500000000000002)

In [111]:
def get_random_weight_or_bias():
    return np.random.normal() - 0.5

class Layer:
    def __init__(self, hidden_size):
        self.hidden_size = hidden_size
    
    def initialize(self):
        self.neurons = [Neuron(get_random_weight_or_bias(), get_random_weight_or_bias(), relu) for _ in range(self.hidden_size)]
    
    def __call__(self, data):
        return np.array([n(data) for n in self.neurons])

In [112]:
l = Layer(10)
l.initialize()
l.neurons

In [113]:
l.neurons[0].weight, l.neurons[0].bias

(0.8444397799585694, 0.17763710715594572)

In [116]:
l(df.iloc[0, :-1])

array([ 8.79092286,  1.58396184,  0.        ,  0.        ,  1.75597419,
       23.74979342,  2.20366733,  0.        ,  0.        ,  8.3610384 ])